In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [3]:
%%writefile mapper2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        key, count = unicode(line.strip()).split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", key, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), count)

Writing mapper2.py


In [6]:
%%writefile reducer2.py

import sys

current_key = None
word_sum = 0
stop_sum=0
list_index=0
percentage=0.0

with open("stop_words_en.txt") as stopfile:
    #stopwords = [line.rstrip() for line in stopfile]
    stopwords = stopfile.readlines()

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    word_sum += count
    for stopword in stopwords:
        #list_index += 1
        #print("%s\t%s" % (stopword.strip(), key))
        if stopword.strip() == key:
            stop_sum += count
            #del stopwords[list_index]
            #break
#print "Words: %d" % word_sum
#print "Stop Words: %d" % stop_sum
word_sum = float(word_sum)
stop_sum = float(stop_sum)
percentage = ( stop_sum / word_sum ) * 100.0
print "%f" % percentage

Overwriting reducer2.py


In [7]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming Stop Words1" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

#hdfs dfs -cat ${OUT_DIR}/part-00000 | head

OUT_DIR2="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming Stop Words2" \
    -D mapreduce.job.reduces=${NUM_REDUCERS2} \
    -files mapper2.py,reducer2.py,/datasets/stop_words_en.txt \
    -mapper "python2 mapper2.py" \
    -reducer "python2 reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null
    
hdfs dfs -cat ${OUT_DIR2}/part-00000

41.602824	


rm: `wordcount_result_1513078572194714': No such file or directory
17/12/12 11:36:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/12 11:36:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/12 11:36:17 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/12 11:36:17 INFO mapreduce.JobSubmitter: number of splits:2
17/12/12 11:36:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1513013126012_0003
17/12/12 11:36:17 INFO impl.YarnClientImpl: Submitted application application_1513013126012_0003
17/12/12 11:36:17 INFO mapreduce.Job: The url to track the job: http://4b76fe1cdd03:8088/proxy/application_1513013126012_0003/
17/12/12 11:36:17 INFO mapreduce.Job: Running job: job_1513013126012_0003
17/12/12 11:36:23 INFO mapreduce.Job: Job job_1513013126012_0003 running in uber mode : false
17/12/12 11:36:23 INFO mapreduce.Job:  map 0% reduce 0%
17/12/12 11:36:40 INFO mapreduce.Job:  map 30% reduce 0%
17/12/12 11:3